In [1]:
# Import necessary modules
import sys
import os

# Set root folder to project root
os.chdir(os.path.dirname(os.getcwd()))

# Add root folder to path
sys.path.append(os.getcwd())

from src.utils.config import Hyperparameters
from src.utils.model_loading import load_model
from src.utils.maze_loading import load_mazes
from src.utils.testing import is_correct
from src.utils.analysis import plot_mazes

In [2]:
# Load model
dt_net_original = load_model('dt_net')
dt_net = load_model('models/dt_net/2025-03-12_03:43:44/best.pth')
it_net = load_model('models/it_net/2025-03-18_17:52:53/best.pth')
model = it_net

# Load mazes
hyperparams = Hyperparameters()
hyperparams.iters = 15
hyperparams.num_mazes = 10
hyperparams.percolation = 0.5
hyperparams.maze_size = 29
inputs, solutions = load_mazes(hyperparams)

# Predict
predictions = model.predict(inputs, iters=hyperparams.iters)

# Evaluate predictions
corrects = is_correct(inputs, predictions, solutions)
print(f'{corrects.sum()} out of {len(corrects)} predictions are correct.')

# Plot results
plot_mazes(
    inputs,
    predictions,
    solutions,
    file_name=f'outputs/visuals/mazes/predictions_{model.name}_size-{hyperparams.maze_size}_iters-{hyperparams.iters}'
)

2025-03-19 14:33:55,947 - src.utils.model_loading - INFO - Loaded dt_net to cuda:2
2025-03-19 14:33:55,955 - src.utils.model_loading - INFO - Loaded models/dt_net/2025-03-12_03:43:44/best.pth to cuda:2
2025-03-19 14:33:55,968 - src.utils.model_loading - INFO - Loaded models/it_net/2025-03-18_17:52:53/best.pth to cuda:2
2025-03-19 14:33:55,969 - src.utils.maze_loading - INFO - Attempting 10 mazes to generate 10 mazes with size: 29, percolation: 0.5, and deadend_start: True
2025-03-19 14:33:55,993 - src.utils.maze_loading - INFO - Loaded 10 mazes with size: 29, percolation: 0.5, and deadend_start: True


1 out of 10 predictions are correct.
